# Symmetry Adapted PDF Analysis - Running SAPA









First, import the sapa library

In [1]:
import sys
sys.path.append('..')

In [ ]:
import sapa as s

Next, we need to create a SAPA object. This imports all the data from a CIF.

In [3]:
struc = s.sapa('BaTiO3_iso.cif')

We can see the list of "methods" the SAPA object has:

In [4]:
dir(struc)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'atom_site_Wyckoff_label',
 'atom_site_fract_symmform',
 'atom_site_fract_x',
 'atom_site_fract_y',
 'atom_site_fract_z',
 'atom_site_label',
 'atom_site_occupancy',
 'atom_site_symmetry_multiplicity',
 'atom_site_type_symbol',
 'cell_angle_alpha',
 'cell_angle_beta',
 'cell_angle_gamma',
 'cell_length_a',
 'cell_length_b',
 'cell_length_c',
 'cell_volume',
 'cleanup',
 'cleanup_single',
 'constraint_array',
 'constraints_out',
 'create_hdf5',
 'create_hdf5_base',
 'create_hdf5_normalise',
 'create_hdf5_occ',
 'create_hdf5_single',
 'execute',
 'execute_occ',
 'execute_old',
 'execute_single',
 'file_chec

A lot of these are variables imported from the CIF itself. For example:

In [5]:
print(struc.cell_length_a)
print(struc.cell_length_b)
print(struc.cell_length_c)

8.07084
8.07084
8.07084


This gives the lattice parameters of the supercell we chose when we generated the CIF with ISODISTORT.

Other methods operate on the information provided in the CIF. For example:

In [6]:
struc.irrep_list()

['GM4-',
 'R4-',
 'X3-',
 'X5-',
 'M3-',
 'M5-',
 'R5+',
 'X1+',
 'X5+',
 'M2-',
 'GM5-',
 'R2-',
 'R3-',
 'R5-',
 'X2+',
 'M1+',
 'M2+',
 'M3+',
 'M4+',
 'M5+']

This provides a list of irreps contained within the CIF. 

We are going to use the "write_inp" method to make an input file to perform symmetry adapted PDF analysis. The required arguments takes are (in order):

sample: 
A "sample" string to remind you what you are running. This is often just the material you are analysing. This is repeated in the output files SAPA produces.

filenameformat: 
This is a string to allow Topas to iterate over the data files. This includes any path information - SAPA assumes the data is in the working directory, so you need to specify if it isn't. Your data file names should be in a format to allow for iteration, i.e. they have a consistent naming convention such as "BaTiO3_15K.xy", "BaTiO3_150K.xy". The temperatures here can be iterated over, and to tell Topas that, replace the temperature with ##n## (filenameformat = "BaTiO3_##n##K.xy")

isneutron:
Set this to True if you are using neutron data, or False for X-ray data. For this example with BaTiO3, we are using neutron data.

qmax:
The maximum value of Q used to generate the PDF data. For this data, qmax = 40.

dq:
The Q-space dampening parameter. For this data, dq = 0.033

startx, finishx:
The r-range to start and end the refinement at. For SAPA, most of the information is contained within the low-r region. For this data, set startx = 1.7 and finishx = 10.0

lattice:
This is a flag to allow lattice parameters to refine as they should for a given lattice type. This can be "C", "T", "O", "R", "H", "TC" or "M", for cubic, tetragonal, orthorhombic, rhombohedral, hexagonal, triclinic and monoclinic, respectively. For now, use lattice = "C".

There are further optional arguments that can be specified. The relevant ones for today are:

cycles:
The number of cycles to perform for each irrep. The default is 300, but for time purposes, lets lower this to 100.

filename:
The default is "batch_modes.inp", but this can be changed. Obviously, it still requires the .inp file extension, whatever the name is. 

In [7]:
struc.write_inp(sample="BaTiO3", filenameformat="BaTiO3_##n##K.xy", isneutron=True, qmax=40.0, dq=0.033, startx=1.7, finishx=10.0, lattice="C", cycles=100)

Creating input file...
120
120
...batch_modes.inp written


This has printed the number of modes in the input file (120 in this case), and has indicated an input file has been written. This can be viewed using jEdit, and we can go through the components of that input file later, time providing.

Now, we set the temperatures of our data files. (NB: these don't have to be temperatures, it is simply something to iterate over). This can be specified as a list (as below) if you don't have a regular interval between data points. Often at I15-1, we collect data at regular intervals, so we could use temps = range(start, finish + interval, interval)

In [8]:
temps = [15, 150, 210, 250, 293, 350, 410, 500]

And now, we can set the code to be run. For each temperature and irrep, this tells Topas to execute the input file with the correct modes activated. Note that this will take a long time to execute. Locking your computer, or letting it go to sleep will interrupt this process, so set the power settings to not go to sleep.

In [9]:
struc.execute(temps)

Finding Topas executables...
Creating Monitoring File...
Executing...
Executing irrep GM4- for temp 15
Completed irrep GM4- for temp 15. Process took 11.921932935714722s (0.19869888226191204 minutes)
Estimated 0.5530452222956551 hours to completion.
Executing irrep R4- for temp 15
Completed irrep R4- for temp 15. Process took 14.222392082214355s (0.23703986803690594 minutes)
Estimated 0.6027719379133648 hours to completion.
Executing irrep X3- for temp 15


C:\workingdirs\sapa_2024\sapa-main\Notebook_Tutorials\BaTiO3\..\sapa_wrk.py:17: SyntaxWarning: invalid escape sequence '\w'
  """A class used to generate and execute symmetry-adapted PDF analysis (SAPA) input files for TOPAS
C:\workingdirs\sapa_2024\sapa-main\Notebook_Tutorials\BaTiO3\..\sapa_wrk.py:217: SyntaxWarning: invalid escape sequence '\]'
  m = re.search("\](.+?)\(", i)
C:\workingdirs\sapa_2024\sapa-main\Notebook_Tutorials\BaTiO3\..\sapa_wrk.py:223: SyntaxWarning: invalid escape sequence '\]'
  m = re.search("\](.+?)\(", i)
C:\workingdirs\sapa_2024\sapa-main\Notebook_Tutorials\BaTiO3\..\sapa_wrk.py:325: SyntaxWarning: invalid escape sequence '\]'
  m = re.search("\](.+?)\(", iso_label[i])
C:\workingdirs\sapa_2024\sapa-main\Notebook_Tutorials\BaTiO3\..\sapa_wrk.py:337: SyntaxWarning: invalid escape sequence '\]'
  m = re.search("\](.+?)\(", iso_label[i])
C:\workingdirs\sapa_2024\sapa-main\Notebook_Tutorials\BaTiO3\..\sapa_wrk.py:1062: SyntaxWarning: invalid escape sequence '\]'

KeyboardInterrupt: 

Completed irrep R5- for temp 410. Process took 87.25986766815186s (1.4543311278025308 minutes)
Estimated 0.9156648288038042 hours to completion.
Executing irrep X2+ for temp 410
Completed irrep X2+ for temp 410. Process took 87.69503545761108s (1.461583924293518 minutes)
Estimated 0.8813649975492599 hours to completion.
Executing irrep M1+ for temp 410
Completed irrep M1+ for temp 410. Process took 70.38759922981262s (1.1731266538302103 minutes)
Estimated 0.846324899025926 hours to completion.
Executing irrep M2+ for temp 410
Completed irrep M2+ for temp 410. Process took 88.27447867393494s (1.4712413112322489 minutes)
Estimated 0.8123700481657844 hours to completion.
Executing irrep M3+ for temp 410
Completed irrep M3+ for temp 410. Process took 62.20978665351868s (1.0368297775586446 minutes)
Estimated 0.7773395286334885 hours to completion.
Executing irrep M4+ for temp 410
Completed irrep M4+ for temp 410. Process took 71.80842351913452s (1.1968070586522421 minutes)
Estimated 0.74297

Occasionally, things can interrupt the refinement process, or cause it to hang (e.g. locking your computer). The execute script writes a file to keep track of the temperature and irrep combinations it has and hasn't ran so far, and this can be used to restart the refinement.

In [ ]:
#struc.restart() 

## Part 2 - Post Execution

Sometimes something can go wrong with an individual irrep/temp combo, but not interrupt the overall SAPA run. An indicator of this is that the ycalc and yobs files for these runs haven't been generated. Missing files causes problems with the HDF5 file creation. The following script checks that all the expected files are there, and if any are found to be missing, it re-runs them.

In [ ]:
struc.file_check(temps, cycles=100, verbose=False) #re-run a couple times to make sure

SAPA generates a large amount of files, which can be quite cumbersome to deal with. To make dealing with SAPA data more straightforward, we place all the data in these files into a structured file format (HDF5). The file is structured in the same way for every SAPA run, meaning analysis is more repeatable.

In [ ]:
struc.create_hdf5(temps)

We are still left with the large number of files however. The following script removes them and (optionally) compresses them into an archive.

In [ ]:
struc.cleanup() # optionally set create_zip = False